In [2]:
#pip install roboflow
#pip install --upgrade pip


from roboflow import Roboflow
rf = Roboflow(api_key="JG5GARYM8y0ghghBbeAk")
project = rf.workspace("sahajprojects").project("omega-plantheight")
model = project.version(1).model


import cv2
import json

def calculate_pixel_height(bbox):
    return bbox['height']

def establish_conversion_factor(reference_height, reference_pixel_height):
    return reference_height / reference_pixel_height

def process_object_detection_results(json_path, image_path, output_path):
    # Load the JSON results
    with open(json_path, 'r') as file:
        detection_results = json.load(file)

    # Extract bounding box information for the reference object (stand) and all plants
    stand_bbox = next(item for item in detection_results['predictions'] if item['class'] == 'stand')
    plant_bboxes = [item for item in detection_results['predictions'] if item['class'] == 'plants']

    # Calculate pixel height of the reference object (stand)
    stand_pixel_height = calculate_pixel_height(stand_bbox)

    # Known height of the reference object (stand) in inches
    stand_height_inches = 60

    # Establish pixel-to-inch conversion factor for the stand
    stand_conversion_factor = establish_conversion_factor(stand_height_inches, stand_pixel_height)

    # Load the original image
    image = cv2.imread(image_path)

    # Draw stand bounding box in green
    stand_box = [
        round(stand_bbox['x'] - (stand_bbox['width']/2)),
        round(stand_bbox['y'] - (stand_bbox['height']/2)),
        round(stand_bbox['x'] + (stand_bbox['width']/2)),
        round(stand_bbox['y'] + (stand_bbox['height']/2))
    ]
    cv2.rectangle(image, (stand_box[0], stand_box[1]), (stand_box[2], stand_box[3]), (0, 255, 0), 2)

    # Iterate through each plant bounding box
    for plant_bbox in plant_bboxes:
        # Calculate the actual height of the plant in inches
        plant_pixel_height = calculate_pixel_height(plant_bbox)
        plant_height_inches = plant_pixel_height * stand_conversion_factor

        # Draw plant bounding box in red
        plant_box = [
            round(plant_bbox['x'] - (plant_bbox['width']/2)),
            round(plant_bbox['y'] - (plant_bbox['height']/2)),
            round(plant_bbox['x'] + (plant_bbox['width']/2)),
            round(plant_bbox['y'] + (plant_bbox['height']/2))
        ]
        cv2.rectangle(image, (plant_box[0], plant_box[1]), (plant_box[2], plant_box[3]), (0, 0, 255), 2)

        # Display plant height label
        font = cv2.FONT_HERSHEY_SIMPLEX
        label_y_offset = 20
        cv2.putText(image, f"Plant Height: {plant_height_inches:.2f} inches", (plant_box[0], plant_box[1] - label_y_offset), font, 0.5, (0, 0, 255), 2, cv2.LINE_AA)

    # Calculate the actual height of the stand in inches
    stand_height_inches = stand_pixel_height * stand_conversion_factor

    # Display stand height label
    font = cv2.FONT_HERSHEY_SIMPLEX
    label_y_offset = 20
    cv2.putText(image, f"Stand Height: {stand_height_inches:.2f} inches", (stand_box[0], stand_box[1] - label_y_offset), font, 0.5, (0, 255, 0), 2, cv2.LINE_AA)

    # Save the annotated image
    cv2.imwrite(output_path, image)
    print(f"Annotated image saved to {output_path}")

# Specify the path to your JSON file, the original image, and the output image
json_path = "C:\\PROOOO\\detection_results777.json"
image_path = "C:\\PROOOO\\test.jpeg"
output_image_path = "C:\\PROOOO\\output_annotated_000.jpg"

# Call the function to process object detection results, calculate heights, and save annotated image
process_object_detection_results(json_path, image_path, output_image_path)


loading Roboflow workspace...
loading Roboflow project...
Annotated image saved to C:\PROOOO\output_annotated_000.jpg
